<a href="https://colab.research.google.com/github/jakartaresearch/adi-buzzer/blob/dev/notebook/R2_convert_data_format_n_get_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
import json
import glob
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive/"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive

In [ ]:
in_path = 'drive/My Drive/Colab Notebooks/adi-buzzer/data/*.pkl'
out_path = 'drive/My Drive/Colab Notebooks/adi-buzzer/data/json/'
path = 'drive/My Drive/Colab Notebooks/adi-buzzer/data/json/*.json'

In [ ]:
def convert(in_path, out_path):
    with open(in_path,'rb') as file:
        data = pickle.load(file)

    for idx in range(len(data['retweets'])):
        data['retweets'][idx] = data['retweets'][idx]._json
    for idx in range(len(data['replies'])):
        data['replies'][idx] = data['replies'][idx]._json
    for idx in range(len(data['tweets'])):
        data['tweets'][idx] = data['tweets'][idx]._json

    username = in_path.split('/')[-1][:-4]
    print(username+'\n')

    with open(out_path+username+'.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

In [ ]:
def get_activity_time(path):
    with open(path, 'r') as f:
        data = json.load(f)

    username = path.split('/')[-1][:-5]
    if (len(data['tweets'])>0) & (len(data['retweets'])>0) & (len(data['replies'])>0):
        end_tweet = to_datetime(data['tweets'][0]['created_at']) 
        start_tweet = to_datetime(data['tweets'][-1]['created_at'])
        
        end_retweet = to_datetime(data['retweets'][0]['created_at'])
        start_retweet = to_datetime(data['retweets'][-1]['created_at'])
        
        end_replies = to_datetime(data['replies'][0]['created_at'])
        start_replies = to_datetime(data['replies'][-1]['created_at'])

        status_datetime = [end_tweet,start_tweet,end_retweet,start_retweet,end_replies,start_replies]
    else :
        status_datetime = [0,0,0,0]
        
    return {'username':username,'start_activity':min(status_datetime), 'end_activity':max(status_datetime)}

## Get all username activity

In [ ]:
all_username_activity = []

for user_path in tqdm(glob.glob(path)):
    all_username_activity.append(get_activity_time(user_path))

In [ ]:
with open(out_path+'activity.pkl', 'wb') as f:
    pickle.dump(all_username_activity, f)

## Convert all data from pickle to json format

In [ ]:
for user_path in tqdm(glob.glob(in_path)):
    convert(user_path, out_path)